In [1]:
import cv2
import matplotlib.pyplot as plt
import os
from PIL import Image
import glob
os.getcwd()

import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
#from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict

%matplotlib inline
np.random.seed(1)

import numpy as np
import keras 
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Permute
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils import to_categorical
#import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline


from numpy import genfromtxt

C:\Users\Marakhi\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
A = 0
k = 0
for filename in glob.glob('C:\\Users\\Marakhi\\Desktop\\Deep learning ajay bhammar\\Deep learning exercise\\Data detection\\raw data\\*.JPG'):
    im=cv2.imread(filename)
    x = im.shape
    height = list(range(x[0]))
    height = height[0:x[0]:int((x[0]-1)/4)]
    width = list(range(x[1]))
    width = width[0:x[1]:int((x[1]-1)/3)]
    print("width, height : ",x[0], x[1], width, height)
    for i in range(3):
        for j in range(4):
            crop_img = im[height[j]:height[j+1], width[i]:width[i+1]]
            resize_img = cv2.resize(crop_img, (40,40), interpolation = cv2.INTER_AREA)
            file_name = 'C:\\Users\\Marakhi\\Desktop\\Deep learning ajay bhammar\\Deep learning exercise\\Data detection\\data\\'+str(A)+'.jpg'
            cv2.imwrite(file_name, resize_img)
            if k==0:
                x_train=np.expand_dims(resize_img, axis=0)
                k = k+1
            else:
                x_train = np.concatenate((x_train, np.expand_dims(resize_img, axis=0)), axis=0)               
            print(width[i+1],height[j+1])
            A = A+1 


width, height :  3033 2129 [0, 709, 1418, 2127] [0, 758, 1516, 2274, 3032]
709 758
709 1516
709 2274
709 3032
1418 758
1418 1516
1418 2274
1418 3032
2127 758
2127 1516
2127 2274
2127 3032
width, height :  2745 2009 [0, 669, 1338, 2007] [0, 686, 1372, 2058, 2744]
669 686
669 1372
669 2058
669 2744
1338 686
1338 1372
1338 2058
1338 2744
2007 686
2007 1372
2007 2058
2007 2744
width, height :  2905 2057 [0, 685, 1370, 2055] [0, 726, 1452, 2178, 2904]
685 726
685 1452
685 2178
685 2904
1370 726
1370 1452
1370 2178
1370 2904
2055 726
2055 1452
2055 2178
2055 2904
width, height :  2681 1889 [0, 629, 1258, 1887] [0, 670, 1340, 2010, 2680]
629 670
629 1340
629 2010
629 2680
1258 670
1258 1340
1258 2010
1258 2680
1887 670
1887 1340
1887 2010
1887 2680
width, height :  2753 1953 [0, 650, 1300, 1950] [0, 688, 1376, 2064, 2752]
650 688
650 1376
650 2064
650 2752
1300 688
1300 1376
1300 2064
1300 2752
1950 688
1950 1376
1950 2064
1950 2752
width, height :  2769 1985 [0, 661, 1322, 1983] [0, 692, 138

In [3]:
y_train = genfromtxt('C:\\Users\\Marakhi\\Desktop\\Deep learning ajay bhammar\\Deep learning exercise\\Data detection\\y_train1.csv', delimiter=',')

In [4]:
print(y_train[0])

[1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.9 5.
 0.9 5.  5.  5.  5.  5.  5.  5.  0.9 5.  0.7 5.  5.  5.  0.8 5.  0.9 5.
 5.  5.  5.  5.  5.  5.  0.9 5.  0.  5.  5.  5. ]


In [6]:
print(y_train[0,0])
x_train1 = x_train[:116]
print(x_train1.shape)
y_train1 = y_train
# y_train = y_train1[:,0:32]
print(y_train1.shape)
#plt.imshow(x_train[242])

1.0
(116, 40, 40, 3)
(116, 48)


In [7]:
#def HappyModel(input_shape):
X_input = Input(x_train1.shape[1:])

X = ZeroPadding2D((3, 3))(X_input)

X = Conv2D(32, (5, 5), strides = (1, 1), name = 'conv0')(X)
X = BatchNormalization(axis = 3, name = 'bn0')(X)
X = Activation('relu')(X)


X = Conv2D(64, (3, 3), strides = (1, 1), name = 'conv1')(X)
X = BatchNormalization(axis = 3, name = 'bn1')(X)
X = Activation('relu')(X)

X = MaxPooling2D((2, 2), name='max_pool1')(X)

X = Conv2D(96, (5, 5), strides = (1, 1), name = 'conv2')(X)
X = BatchNormalization(axis = 3, name = 'bn2')(X)
X = Activation('relu')(X)

# X = Conv2D(96, (3, 3), strides = (1, 1), name = 'conv3')(X)
# X = BatchNormalization(axis = 3, name = 'bn3')(X)
# X = Activation('relu')(X)

# X = Conv2D(128, (5, 5), strides = (1, 1), name = 'conv4')(X)
# X = BatchNormalization(axis = 3, name = 'bn4')(X)
# X = Activation('relu')(X)

# X = Conv2D(160, (3, 3), strides = (1, 1), name = 'conv5')(X)
# X = BatchNormalization(axis = 3, name = 'bn5')(X)
# X = Activation('relu')(X)

# X = Conv2D(192, (5, 5), strides = (1, 1), name = 'conv6')(X)
# X = BatchNormalization(axis = 3, name = 'bn6')(X)
# X = Activation('relu')(X)


# X = Conv2D(128, (3, 3), strides = (1, 1), name = 'conv3')(X)
# X = BatchNormalization(axis = 3, name = 'bn3')(X)
# X = Activation('relu')(X)

# X = Conv2D(128, (3, 3), strides = (1, 1), name = 'conv4')(X)
# X = BatchNormalization(axis = 3, name = 'bn4')(X)
# X = Activation('relu')(X)

X = MaxPooling2D((2, 2), name='max_pool2')(X)

X = Conv2D(256, (5, 5), strides = (1, 1), name = 'conv5')(X)
X = BatchNormalization(axis = 3, name = 'bn5')(X)
X = Activation('relu')(X)

# X = Conv2D(384, (3, 3), strides = (1, 1), name = 'conv6')(X)
# X = BatchNormalization(axis = 3, name = 'bn6')(X)
# X = Activation('relu')(X)

X = Conv2D(3, (1, 1), strides = (1, 1), name = 'conv7')(X)

X = Permute((3, 1, 2))(X)
Y = Flatten()(X)

model = Model(inputs = X_input, outputs = Y, name='HappyModel')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 40, 3)         0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 46, 46, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 42, 42, 32)        2432      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 42, 42, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 42, 42, 32)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 40, 40, 64)        18496     
_________________________________________________________________
bn1 (BatchNormalization)     (None, 40, 40, 64)        256       
__________

In [8]:
def my_loss(y_true, y_pred):
    class_loss = tf.reduce_mean((y_true[:,0:16]-y_pred[:,0:16])**2,)
    cond = tf.equal(tf.squeeze(y_true[:,0:16]),tf.constant([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=tf.float32))
    ind = tf.where(cond)
    val1 = tf.squeeze(tf.gather_nd(y_true[:,16:32],ind))
    val2 = tf.squeeze(tf.gather_nd(y_pred[:,16:32],ind))
    X_loss = tf.reduce_mean((val1-val2)**2,)
    val3 = tf.squeeze(tf.gather_nd(y_true[:,32:48],ind))
    val4 = tf.squeeze(tf.gather_nd(y_pred[:,32:48],ind))
    Y_loss = tf.reduce_mean((val3-val4)**2,)
    loss = class_loss + 3*X_loss + 3*Y_loss
    return loss

In [9]:
model.compile('adam', loss=my_loss)
model.fit(x_train1, y_train1, epochs=60, batch_size=15)

Epoch 1/60
116/116 [==============================] - 9s 80ms/step - loss: 22.3283
Epoch 2/60
116/116 [==============================] - 7s 59ms/step - loss: 2.2377
Epoch 3/60
116/116 [==============================] - 7s 61ms/step - loss: 1.5100
Epoch 4/60
116/116 [==============================] - 7s 57ms/step - loss: 1.0603
Epoch 5/60
116/116 [==============================] - 7s 58ms/step - loss: 0.8720
Epoch 6/60
116/116 [==============================] - 7s 61ms/step - loss: 0.7946
Epoch 7/60
116/116 [==============================] - 7s 58ms/step - loss: 0.7389
Epoch 8/60
116/116 [==============================] - 7s 57ms/step - loss: 0.6878
Epoch 9/60
116/116 [==============================] - 7s 58ms/step - loss: 0.6818
Epoch 10/60
116/116 [==============================] - 7s 57ms/step - loss: 0.6639
Epoch 11/60
116/116 [==============================] - 7s 57ms/step - loss: 0.6720
Epoch 12/60
116/116 [==============================] - 7s 57ms/step - loss: 0.6359
Epoch 13/60


In [21]:
model.save('detection_model.h5')

In [22]:
t = tf.constant([[[1, 1, 1], [2, 2, 2]], [[3, 3, 3], [4, 4, 4]]])
yes = tf.shape(t)
tf.print(yes)

<tf.Operation 'PrintV2' type=PrintV2>

In [96]:
x = image.img_to_array(x_train[81])
x = np.expand_dims(x, axis=0)
#print(x)
#x = preprocess_input(x)
#print(y_train1[82])
mark_y=model.predict(x)
#ma = tf.sigmoid(mark_y)
#print(mark_y[0])
print(mark_y[0][0:16])
print(mark_y[0][16:32])
print(mark_y[0][32:48])

[ 0.33974847 -0.11143951 -0.00190906 -0.19311832 -0.10497604  0.33549157
 -0.15510722 -0.1637186   0.17428674 -0.01604165  0.2756063   0.04511161
  0.14782228  0.2877231   0.11416484 -0.33574146]
[ 0.8745198  -0.20143956 -0.23595184 -0.15796614  0.5839783  -0.05094298
  0.02787497 -0.36368263  0.52246106  0.27963287  0.29898387  0.01148047
  0.63379854  0.36643654  0.4049906  -0.00840219]
[ 0.39552572  0.61212814  0.4520188   0.3767602   0.6763377   0.3982406
  0.4218561   0.46703294  0.36192623  0.34151903  0.52777314  0.12592065
  0.17676681  0.09863904 -0.05818508  0.08046089]


In [16]:
for i in range(123,124,1):
    print(i)
    file_name = 'C:\\Users\\Marakhi\\Desktop\\Deep learning ajay bhammar\\Deep learning exercise\\Data detection\\test\\'+str(i)+'.jpg'
    im = cv2.imread(file_name)
    x = image.img_to_array(im)
    x = np.expand_dims(x, axis=0)
    result = model.predict(x)
    print(result[0][0:16])
    print(result[0][16:32])
    print(result[0][32:48])
    boxes = [[0,0,0,0]]
    scores = []
    for j in range(0,16,1):
        if j == 0:
            XX = 0
            YY = 0
        if j == 1:
            XX = 10
            YY = 0
        if j == 2:
            XX = 20
            YY = 0
        if j == 3:
            XX = 30
            YY = 0  
        if j == 4:
            XX = 0
            YY = 10
        if j == 5:
            XX = 10
            YY = 10
        if j == 6:
            XX = 20
            YY = 10
        if j == 7:
            XX = 30
            YY = 10 
        if j == 8:
            XX = 0
            YY = 20
        if j == 9:
            XX = 10
            YY = 20
        if j == 10:
            XX = 20
            YY = 20
        if j == 11:
            XX = 30
            YY = 20  
        if j == 12:
            XX = 0
            YY = 30
        if j == 13:
            XX = 10
            YY = 30
        if j == 14:
            XX = 20
            YY = 30
        if j == 15:
            XX = 30
            YY = 30
        if result[0][j] > 0.1:
            print(result[0][j])
            if result[0][j+16] > 0.89:
                result[0][j+16] = 0.9
            if result[0][j+16] < 0.01:
                result[0][j+16] = 0
            if result[0][j+32] > 0.89:
                result[0][j+32] = 0.9
            if result[0][j+32] < 0.01:
                result[0][j+32] = 0  
            a = XX+result[0][j+16]*10-6
            b = YY+result[0][j+32]*10-6
            c = XX+result[0][j+16]*10+6
            d = YY+result[0][j+32]*10+6
            box = [[int(b),int(a),int(d),int(c)]]
            boxes = np.concatenate((boxes, box), axis=0)
            print(boxes)
            score = [result[0][j]]
            scores = np.concatenate((scores, score), axis=0)
    boxes = boxes[1:]
    selected_indices = tf.image.non_max_suppression(boxes, scores, 15, 0.05, 0.1)
    print(boxes)
    print(scores)
    with tf.Session() as test_b:
        print(selected_indices.eval())
        for i in selected_indices.eval():
            det = cv2.rectangle(im,(boxes[i][1],boxes[i][0]),(boxes[i][3],boxes[i][2]),(255,255,255),1)
            print(boxes[i][1],boxes[i][0],boxes[i][3],boxes[i][2])
            cv2.imwrite(file_name, det)        
    
#     locliz = cv2.circle(im,(int(tar_y[0][0]*100),int(tar_y[0][1]*100)),13,(255,255,255))
#     cv2.imwrite(file_name, locliz)

123
[ 0.3140617   0.3894261   0.21807307  0.14379394  0.41166842  0.09699273
  0.34146708  0.19907326  0.46319747  0.38818824  0.47714144  0.13080174
  0.04339451  0.06234241  0.38601863 -0.17818385]
[ 0.95952475  0.6002728   0.38905448 -0.32195786  1.1552869   0.55276203
  0.20569828 -0.33810893  1.1003695   0.5413618   0.18624714 -0.10295856
  0.9854902   0.5341711   0.32454687 -0.17737243]
[ 0.9765161   0.8613509   0.3532141   0.5016957   0.29038152  0.43597212
  0.53188634  0.39110115  0.01259077  0.6258584   0.7893013   0.2718267
 -0.24394143  0.2560677  -0.05182055 -0.051632  ]
0.3140617
[[ 0  0  0  0]
 [ 2  2 14 14]]
0.3894261
[[ 0  0  0  0]
 [ 2  2 14 14]
 [ 2 10 14 22]]
0.21807307
[[ 0  0  0  0]
 [ 2  2 14 14]
 [ 2 10 14 22]
 [-2 17  9 29]]
0.14379394
[[ 0  0  0  0]
 [ 2  2 14 14]
 [ 2 10 14 22]
 [-2 17  9 29]
 [ 0 24 11 36]]
0.41166842
[[ 0  0  0  0]
 [ 2  2 14 14]
 [ 2 10 14 22]
 [-2 17  9 29]
 [ 0 24 11 36]
 [ 6  2 18 14]]
0.34146708
[[ 0  0  0  0]
 [ 2  2 14 14]
 [ 2 10 14

In [41]:
happyModel.save('my_model_detection.h5')

In [33]:
for i in [0, 2, 5, 4]:
    print(i)

0
2
5
4
